In [ ]:
# 서울시 공원 지도에 표시하기 

In [4]:
import folium
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib as mpl 
import matplotlib.pyplot as plt 
%matplotlib inline


In [ ]:
mpl.rc('font', family='Malgun Gothic')
mpl.rc('axes', unicode_minus=False)

In [ ]:
### 데이터 파악하고 정리하기 

In [5]:
park = pd.read_csv('서울시 주요 공원현황(수정).csv')
park.head(3)

,공원번호,공원명,공원개요,면적,개원일,주요시설,주요식물,안내도,오시는길,이용시참고사항,이미지,지역,공원주소,관리부서,전화번호,X좌표(GRS80TM),Y좌표(GRS80TM),X좌표(WGS84),Y좌표(WGS84),바로가기
0,1,남산도시자연공원,남산공원은 도심에 위치하여 서울시민에게 맑은 공기를 제공하는 자연휴식처이며 산책 꽃...,2896887㎡ 2454140㎡ 442747㎡,1968.9.10,기반시설 : 광장 45950㎡ 도로 108530㎡ 산책로 6.7㎞ (북측:3.7㎞/...,소나무 단풍 아카시아 상수리나무 등 191종 2881870주,http://parks.seoul.go.kr/upload/seditorMulti/2...,명동역(남산케이블카 와룡묘 서울애니메이션센터 방면) 도보 명동역 3번출구 → 퍼시픽...,남산공원 차량통행 금지안내 2005년 5월 1일부터 남산공원에 일반승용차 택시 통행...,http://parks.seoul.go.kr/file/info/view.do?fId...,중구,서울특별시 중구 삼일대로 231(예장동),서울특별시중부푸른도시사업소,02-3783-5900,198364.107,450395.554,126.990377,37.550140,http://parks.seoul.go.kr/template/sub/namsan.do
1,3,길동생태공원,길동생태공원은 생물의 서식처를 제공하고 종다양성을 증진시키며 자연생태계의 생물들을 ...,80683㎡,1999.5.20,탐방객안내소 야외전시대 관찰대 목재데크 조류관찰대 길동생태문화센터,소나무 보리수 등 64종 31800주 산국 부들 등 138종 192800본,http://parks.seoul.go.kr/template/common/img/p...,NaN,매주 월요일은 휴관입니다 사전예약 후 입장하실 수 있습니다. ▶ 인터넷 예약 바로가...,http://parks.seoul.go.kr/file/info/view.do?fId...,강동구,서울특별시 강동구 천호대로 1291(길동생태공원),동부공원녹지사업소 길동생태공원,02-489-2770,213554.120,448852.675,127.154779,37.540394,http://parks.seoul.go.kr/template/sub/gildong.do
2,4,서울대공원,서울대공원은 세계 각국의 야생동물들이 살아 숨 쉬는 서울동물원과 다양한 재미와 즐거...,9132690㎡,1984.5.1,동물원 식물원 테마가든(장미원 어린이동물원 피크닉장) 치유숲 산림욕장 캠핑장 국립현...,NaN,NaN,지하철 4호선 대공원역 하차(2번 출구) 도보 15분 정도 문의처 02)500-73...,일반사항 공원을 깨끗하게 이용합니다. 대중교통을 이용해 주세요. 기념물 시설물 풀과...,http://parks.seoul.go.kr/file/info/view.do?fId...,과천시,경기도 과천시 대공원광장로 102,전략기획실,02-500-7032,200994.267,437163.981,127.019846,37.426449,http://grandpark.seoul.go.kr/


In [6]:
# null값 확인
park.isnull().sum()

공원번호              0
공원명               0
공원개요              1
면적                3
개원일              10
주요시설              8
주요식물             44
안내도              30
오시는길             17
이용시참고사항          12
이미지               0
지역                0
공원주소              0
관리부서              0
전화번호              0
X좌표(GRS80TM)     10
Y좌표(GRS80TM)     10
X좌표(WGS84)        1
Y좌표(WGS84)        1
바로가기            114
dtype: int64

In [7]:
# 면적 null부분을 0으로 채우기 
park['면적'] = park.면적.fillna('0')
park.면적.isnull().sum()

0

In [ ]:
def calc_area(s):
    if s == '0':
        return 0

    num_list = s.strip().split('㎡')
    area = 0
    for num_str in num_list:
        if num_str == '':
            continue
        area += float(num_str)

    return area

In [ ]:
park['area'] = park.면적.map(calc_area).astype(int)
park[['면적','area']].head()

In [ ]:
park_new = park[['공원명','지역','공원주소','area','공원개요']]
park_new.head()

## 위도, 경도 정보 구하기

In [ ]:
import requests
from urllib.parse import quote

In [ ]:
with open('kakaoapikey.txt', mode='r') as key_file:
    kakao_key = key_file.read()
local_url = 'https://dapi.kakao.com/v2/local/search/address.json'

In [ ]:
lat_list, lng_list = [], []
for i in park_new.index:
    try:
        url = f"{local_url}?query={quote(park_new['공원주소'][i])}"
        result = requests.get(url,
                headers={"Authorization": "KakaoAK "+kakao_key}).json()
        lat = float(result['documents'][0]['y'])
        lng = float(result['documents'][0]['x'])
        lat_list.append(lat)
        lng_list.append(lng)
    except:
        print(park_new.공원명[i], park_new.공원주소[i])

In [ ]:
In [12]:
lat_list, lng_list = [], []
for i in park_new.index:
    try:
        addr = park_new.공원주소[i].split('(우)')[0]
        url = f"{local_url}?query={quote(addr)}"
        result = requests.get(url,
                headers={"Authorization": "KakaoAK "+kakao_key}).json()
        lat = float(result['documents'][0]['y'])
        lng = float(result['documents'][0]['x'])
        lat_list.append(lat)
        lng_list.append(lng)
    except:
        print(park_new.공원명[i], park_new.공원주소[i])

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
park_new['lat'], park_new['lng'] = lat_list, lng_list
park_new.head()

In [ ]:
park_new.area.plot(kind='hist')
plt.show()

In [ ]:
area_criteria = [-1, 100000, 1000000, 12000000]
labels = ['소형', '중형', '대형']
size_info = [3, 7, 15]

In [ ]:
park_new['scale'] = pd.cut(park_new.area, area_criteria, labels=labels)
park_new['size'] = pd.cut(park_new.area, area_criteria, labels=size_info)
park_new.head()

In [ ]:
### 지도에 공원 표시하기 

In [ ]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11)
for i in park_new.index:
    folium.CircleMarker(
        location=[park_new.lat[i], park_new.lng[i]], 
        radius=int(park_new['size'][i]),
        tooltip=f"{park_new.공원명[i]}({park_new.area[i]:,d}㎡)",
        color='#3186cc', fill_color='#3186cc'
    ).add_to(map)

title_html = '<h3 align="center" style="font-size:20px"><b>서울시내 공원 위치와 크기</b></h3>'   
map.get_root().html.add_child(folium.Element(title_html))
map

In [ ]:
### 자치구별 공원 분석 

In [ ]:
park_gu = park_new.groupby('지역').sum()
park_gu['공원수'] = park_new.groupby('지역').count()['area']
del park_gu['lat']
del park_gu['lng']
park_gu.drop(['과천시'], inplace=True)
park_gu.rename(columns={'area': '공원면적'}, inplace=True)
park_gu.head()

In [ ]:
- 구별 공원면적

In [ ]:
import json
geo_str = json.load(open('skorea_municipalities_geo_simple.json',
                         encoding='utf8'))

In [ ]:
map = folium.Map(
        location=[37.5502, 126.982], 
        zoom_start=11, tiles='Stamen Toner'
)
map.choropleth(
    geo_data = geo_str,
    data = park_gu.공원면적,
    columns = [park_gu.index, park_gu.공원면적],
    fill_color = 'PuRd',
    key_on = 'feature.id'
)
for i in park_new.index:
    folium.CircleMarker(
        location=[park_new.lat[i], park_new.lng[i]], 
        radius=int(park_new['size'][i]),
        tooltip=f"{park_new.공원명[i]}({park_new.area[i]:,d}㎡)",
        color='green', fill_color='green'
    ).add_to(map)

title_html = '<h3 align="center" style="font-size:20px"><b>자치구별 공원면적</b></h3>'   
map.get_root().html.add_child(folium.Element(tit